---
>「もし他人の顔を選んだのなら その『顔』は決してあなたにはなじまない」 \
> 辻彩(ジョジョの奇妙な冒険)
---

# はじめに

ここでは、StyleGANに関連した最新応用例をいくつか紹介する

- SemanticStyleGAN(意味領域編集可能StyleGAN)
- DragGAN(対話的Image Manipulation GAN)
- SadTalker(画像のリップシンク)
- Wav2Lip(動画のリップシンク)

まず、Pillow問題を解決する

1. Pillowのバージョンを下げ、さらに、セッションを再起動する
1. そのあと、その次のセルから以降を実行する

この手順を守れば実行可能です


In [16]:
!pip install Pillow==9.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 MB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-9.5.0-cp312-cp312-linux_x86_64.whl size=1210315 sha256=6aa084f963a882f9357e0416369c5e6cb3ee8233c4ead74c04ad7c71bd5962b1
  Stored in directory: /root/.cache/pip/wheels/ea/de/2e/75a6399e5d8cd3a55c13c8f0658d996d4ce4cff37389de044c
Successfully built Pillow
  Attempting uninstall: Pillow
    Found existing installation: pillow 11.3.0
    Uninstalling pillow-11.3.0:
      Successfully uninstalled pillow-11.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-image 0.25.2 requires pillow>=10.1, but you have pillow 9.5.0 which is incompatible.


# SemanticStyleGAN(意味領域編集可能StyleGAN)

SemanticStyleGANは、意味領域ごとの編集が可能なStyleGANであり、セマンティックマスクを用いて、潜在空間を意味領域ごとに分けることが可能な学習フレームワークである

特に、既存の画像編集手法と組み合わせることでより細かい編集が可能になっている

## 特徴

StyleGANの潜在変数の意味は比較的曖昧であり、異なる属性同士が相関しているため、属性操作などの編集をしようとした際に予期しない属性や部位が編集されるという問題がある

この問題に対処しつつ、局所的な操作、例えば顔画像の場合だと目だけ編集したり、髪だけ編集するなどを可能とするのがSemanticStyleGANである

SemanticStyleGANを用いて顔画像を編集した例を示す
- 一番上の行に編集した意味領域と参照画像が示されており、指定している意味領域のみがうまく編集されている
- 他の領域に大きな影響を与えていないことがわかる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssg1.png" width=700>

## 手法

### どのように領域ごとに切り分けるか

領域ごとのローカル生成器を使い、それらを合成することで最終出力を得るような構造にすることで領域ごとに切り分けている

### 領域ごとの意味付けをどのように行うか。

RGB画像とセマンティックマスクのペアを結合したものを識別器の入力とし、RGB画像とセマンティックマスクの結合分布を学習するという手法で意味づけを行っている


## 構造

次の図は学習フレームワークの全体の概要を示している

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssg2.png" width=700>

ローカル生成器gk、Fusionモジュール、レンダリングネットワークR、学習方法は以下の通りである

### ローカル生成器

入力が座標を符号化したフーリエ特徴量と潜在変数wで、深度値dkと特徴量fkを出力する

潜在変数はwはwbase、wsk、wtkに分けられており、これらを分けて入力とすることで、Coarse、Structure、Textureに分けて学習することができ、推論時それぞれの要素の操作が可能となる

最終出力画像のサイズを256x256としているが、ローカル生成器は画素ごとに処理を行うため、フーリエ特徴量をそのまま256x256用意する場合計算量が膨大となる
- そこで性能と計算量のバランスをみて、入力のサイズを小さくしフーリエ特徴量を64x64としている

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssg3.png" width=500>

### Fusionモジュール

ローカル生成器で出力した深度値dkと特徴量fkをFusionモジュールで融合する
- 特徴マップfは、セマンティックマスクmとローカル生成器で出力した特徴量fkのアダマール積であり、(クラス数)x256x256のセマンティックマスクmと特徴マップfが生成される
- レンダリングネットワークRの学習に使われる

### レンダリングネットワークR

レンダリングネットワークRは、StyleGAN2の生成器を改良して利用している
- StyleGAN2のstyle modulationは除外し、入力は特徴マップのみとしている
- 入力の特徴マップは16x16にリサイズしたものを使用しており、広い範囲でのクラス間の特徴を捉えられるようにしている

出力として256x256のRGB画像とセマンティックマスクを得るために出力の分岐を増やしている
- それぞれの分岐では前の段階で出力された画像からの残差を出力する
- 小さい解像度の出力に対し、アップサンプリングと合算を繰り返すことで最終的な解像度の出力を求めている

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssg4.png" width=400>

### 学習
RGB画像とセマンティックマスクの結合分布を学習するためにそれら両方を識別器の入力としている
- 単純に2つを結合して入力するとセマンティックマスクの勾配が大きいためにうまくいかないため、次のような構造の識別器を用いている
- セグメンテーション側のネットワークにおいてR1正則化で勾配にペナルティを与えることで学習を進めることができる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssg5.png" width=400>

最終的なマスクが粗いマスクから大きく外れないようにするため、以下の正則化を損失関数に追加している

$$\mathcal{L}_{all}=\mathcal{L}_{StyleGAN2}+\lambda_{mask}\mathcal{L}_{mask}+\lambda_{R1_{seg}}\mathcal{L}_{R1_{seg}}$$


## 評価

### 潜在空間分離

セマンティックセグメンテーションの領域ごとに分離して画像を生成できているかを確認する
- 領域の成分を徐々に増やしながら生成した結果が以下の画像であり、表示されている擬似深度マップは追加された領域を表す

それぞれの領域が独立して生成できていることがわかる
- 3Dの情報を追加していないが、擬似深度マップは意味のある形状を学習できていることがわかる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/ssg6.png" width=700>

結果として、StyleGAN2を用いた結果では潜在空間のもつれのせいで、編集したい部分以外の無関係な部分まで変更されるケースがあるが、SemanticStyleGANを用いると、領域ごとに潜在空間が分離されているため、無関係なところは変更されず、編集したい部分だけ操作できる


## 動作確認

### 環境セットアップ

### Githubからソースコード取得

In [1]:
%cd /content

!git clone https://github.com/seasonSH/SemanticStyleGAN.git
# for align face
!git clone https://github.com/adamian98/pulse.git

/content
fatal: destination path 'SemanticStyleGAN' already exists and is not an empty directory.
Cloning into 'pulse'...
remote: Enumerating objects: 102, done.
remote: Total 102 (delta 0), reused 0 (delta 0), pack-reused 102 (from 1)
Receiving objects: 100% (102/102), 164.43 MiB | 19.99 MiB/s, done.
Resolving deltas: 100% (46/46), done.


### ライブラリのインストール

以下のセルで、入力が求められる場合があるが、その場合はクリックして、Yesと答えるとよい

また、セッションの再起動が求められる場合は、再起動して、次のセルから実行すること

In [3]:
%cd /content/SemanticStyleGAN

# ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip > /dev/null
!sudo unzip ninja-linux.zip -d /usr/local/bin/ > /dev/null
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force > /dev/null

# Install other requirements first
!pip install -r requirements.txt

# Explicitly downgrade Pillow to a compatible version AFTER other requirements
# This version is necessary for imageio.imwrite to avoid the _Tile AttributeError.
!pip install Pillow==9.5.0

print("Pillow has been set to 9.5.0. Please restart the runtime (Runtime -> Restart runtime) for changes to take full effect.")
# from google.colab import runtime
# runtime.unassign() # Uncomment this line and run this cell to automatically restart the runtime

/content/SemanticStyleGAN
--2025-12-11 02:50:37--  https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/1335132/d2f252e2-9801-11e7-9fbf-bc7b4e4b5c83?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-11T03%3A41%3A51Z&rscd=attachment%3B+filename%3Dninja-linux.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-11T02%3A41%3A00Z&ske=2025-12-11T03%3A41%3A51Z&sks=b&skv=2018-11-09&sig=zJjQ17poJnJzJ4Q8KFQ17Tg9inxhcNr65%2BiK7yjw550%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTQyMTczNywibmJmIjoxNzY1NDIxNDM3LCJwYXRoIjoicmVsZWFzZW

Pillow has been set to 9.5.0. Please restart the runtime (Runtime -> Restart runtime) for changes to take full effect.


### ライブラリのインポート

In [1]:
%cd /content/SemanticStyleGAN

import os
import argparse
import shutil
import numpy as np
import imageio
import time
import torch

from models import make_model
from visualize.utils import generate, cubic_spline_interpolate

from criteria.lpips import lpips

/content/SemanticStyleGAN


### 学習済みモデルのセットアップ

In [2]:
%cd /content/SemanticStyleGAN
!mkdir pretrained

if not os.path.exists('./pretrained/CelebAMask-HQ-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt \
        -O ./pretrained/CelebAMask-HQ-512x512.pt
if not os.path.exists('./pretrained/BitMoji-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/BitMoji-512x512.pt \
        -O ./pretrained/BitMoji-512x512.pt
if not os.path.exists('./pretrained/MetFaces-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/MetFaces-512x512.pt \
        -O ./pretrained/MetFaces-512x512.pt
if not os.path.exists('./pretrained/Toonify-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/Toonify-512x512.pt \
        -O ./pretrained/Toonify-512x512.pt

/content/SemanticStyleGAN
--2025-12-11 02:52:49--  https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/501463958/53b0a617-7635-41f5-910e-cc255dbcd0f7?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-11T03%3A43%3A02Z&rscd=attachment%3B+filename%3DCelebAMask-HQ-512x512.pt&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-11T02%3A42%3A55Z&ske=2025-12-11T03%3A43%3A02Z&sks=b&skv=2018-11-09&sig=L6cFllWHNFrZLQnrxaQT%2B0Yg2fARM7W2nQRurpwTdv0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTQyNTE2OSwibmJmIjoxNzY1NDI

### Random Synthesis

In [3]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/CelebAMask-HQ-512x512.pt'
args.outdir = './results/samples'
args.batch = 8
args.sample = 20
args.truncation = 0.7
args.truncation_mean = 10000
args.save_latent = True
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt, weights_only=False)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating images ...")
start_time = time.time()
with torch.no_grad():
  styles = model.style(torch.randn(args.sample, model.style_dim, device=args.device))
  styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  images, segs = generate(model, styles, mean_latent=mean_latent, batch_size=args.batch)
  for i in range(len(images)):
    # Pillow is now globally set to 9.5.0 after runtime restart. No need to reinstall in loop.
    imageio.imwrite(f"{args.outdir}/{str(i).zfill(6)}_img.jpg", images[i])
    imageio.imwrite(f"{args.outdir}/{str(i).zfill(6)}_seg.jpg", segs[i])
    if args.save_latent:
      np.save(f'{args.outdir}/{str(i).zfill(6)}_latent.npy', styles[i:i+1].cpu().numpy())
print(f"Average speed: {(time.time() - start_time)/(args.sample)}s")

/content/SemanticStyleGAN
Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 8,
 'channel_multiplier': 2,
 'checkpoint_dir': 'output/celeba-512',
 'ckpt': None,
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': 'dataset/lmdb_celebamaskhq_512',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'g_reg_every': 4,
 'inception': 'cache/cache_fid_celebahq_512_new.pkl',
 'iter': 800000,
 'lambda_mask': 100.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 4,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1_img': 10,
 'r1_seg': 1000,
 'residual_refine': True,
 'save_every': 10000,
 'seg_dim': 13,
 'size': 512,
 'start_iter': 20000,
 'transparent_dims': (10, 12),
 'viz_every': 2000}
n_latent: 28, n_latent_expand: 130
Generating images ...


/content/SemanticStyleGAN/models/op/conv2d_gradfix.py:96: UserWarning: conv2d_gradfix not supported on PyTorch 2.9.0+cu126. Falling back to torch.nn.functional.conv2d().
  warnings.warn(


Average speed: 0.17108047008514404s


### Generate Video

In [4]:
latent_dict_celeba = {
  2:  "bcg_1",
  3:  "bcg_2",
  4:  "face_shape",
  5:  "face_texture",
  6:  "eye_shape",
  7:  "eye_texture",
  8:  "eyebrow_shape",
  9:  "eyebrow_texture",
  10: "mouth_shape",
  11: "mouth_texture",
  12: "nose_shape",
  13: "nose_texture",
  14: "ear_shape",
  15: "ear_texture",
  16: "hair_shape",
  17: "hair_texture",
  18: "neck_shape",
  19: "neck_texture",
  20: "cloth_shape",
  21: "cloth_texture",
  22: "glass",
  24: "hat",
  26: "earing",
  0:  "coarse_1",
  1:  "coarse_2",
}

In [5]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/CelebAMask-HQ-512x512.pt'
args.latent = './results/samples/000000_latent.npy'
args.outdir = './results/videos'
args.batch = 8
args.sample = 10
args.steps = 160
args.truncation = 0.7
args.truncation_mean = 10000
args.dataset_name = 'celeba'
args.device = 'cuda'


if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt, weights_only=False)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])
print("Generating videos ...")
if args.dataset_name == "celeba":
  latent_dict = latent_dict_celeba
else:
  raise ValueError("Unknown dataset name: f{args.dataset_name}")

with torch.no_grad():
  for latent_index, latent_name in latent_dict.items():
    styles_new = styles.repeat(args.sample, 1, 1)
    mix_styles = model.style(torch.randn(args.sample, 512, device=args.device))
    mix_styles[-1] = mix_styles[0]
    mix_styles = args.truncation * mix_styles + (1-args.truncation) * mean_latent.unsqueeze(0)
    mix_styles = mix_styles.unsqueeze(1).repeat(1,model.n_latent,1)
    styles_new[:,latent_index] = mix_styles[:,latent_index]
    styles_new = cubic_spline_interpolate(styles_new, step=args.steps)
    images, segs = generate(model, styles_new, mean_latent=mean_latent, randomize_noise=False, batch_size=args.batch)
    frames = [np.concatenate((img,seg),1) for (img,seg) in zip(images,segs)]
    imageio.mimwrite(f'{args.outdir}/{latent_index:02d}_{latent_name}.mp4', frames, fps=20)
    print(f"{args.outdir}/{latent_index:02d}_{latent_name}.mp4")

/content/SemanticStyleGAN
Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 8,
 'channel_multiplier': 2,
 'checkpoint_dir': 'output/celeba-512',
 'ckpt': None,
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': 'dataset/lmdb_celebamaskhq_512',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'g_reg_every': 4,
 'inception': 'cache/cache_fid_celebahq_512_new.pkl',
 'iter': 800000,
 'lambda_mask': 100.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 4,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1_img': 10,
 'r1_seg': 1000,
 'residual_refine': True,
 'save_every': 10000,
 'seg_dim': 13,
 'size': 512,
 'start_iter': 20000,
 'transparent_dims': (10, 12),
 'viz_every': 2000}
n_latent: 28, n_latent_expand: 130
Generating original image ...
Generating vid

### videoの表示

In [6]:
!pip install moviepy

In [7]:
%cd /content/SemanticStyleGAN

from moviepy.editor import *
from moviepy.video.fx.resize import resize

concat_video = './results/videos/concat.mp4'

clip1 = VideoFileClip('/content/SemanticStyleGAN/results/videos/04_face_shape.mp4')
clip2 = VideoFileClip('/content/SemanticStyleGAN/results/videos/06_eye_shape.mp4')
clip3 = VideoFileClip('/content/SemanticStyleGAN/results/videos/10_mouth_shape.mp4')
clip4 = VideoFileClip('/content/SemanticStyleGAN/results/videos/18_neck_shape.mp4')
clip5 = VideoFileClip('/content/SemanticStyleGAN/results/videos/16_hair_shape.mp4')
clip6 = VideoFileClip('/content/SemanticStyleGAN/results/videos/17_hair_texture.mp4')

concat = clips_array([[clip1, clip2], [clip3, clip4], [clip5, clip6]])
concat.write_videofile(concat_video)

/content/SemanticStyleGAN
Moviepy - Building video ./results/videos/concat.mp4.
Moviepy - Writing video ./results/videos/concat.mp4



Moviepy - Done !
Moviepy - video ready ./results/videos/concat.mp4


In [8]:
concat = resize(concat, width=640)
concat.ipython_display()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4


### 潜在ベクトルを求める(invert)

In [9]:
%cd /content/SemanticStyleGAN
!rm -rf test_img
!mkdir -p test_img/src test_img/align

!wget -c https://www.pakutaso.com/shared/img/thumb/kuchikomi725_TP_V.jpg \
      -O ./test_img/src/test1.jpg

%cd /content/pulse
# Fix for AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'
# PIL.Image.ANTIALIAS has been deprecated. Use PIL.Image.LANCZOS instead.
!sed -i "s/PIL.Image.ANTIALIAS/PIL.Image.LANCZOS/g" shape_predictor.py

!python align_face.py \
  -input_dir /content/SemanticStyleGAN/test_img/src \
  -output_dir /content/SemanticStyleGAN/test_img/align \
  -output_size 512 \
  -seed 12

/content/SemanticStyleGAN
--2025-12-11 03:06:04--  https://www.pakutaso.com/shared/img/thumb/kuchikomi725_TP_V.jpg
Resolving www.pakutaso.com (www.pakutaso.com)... 133.242.30.52
Connecting to www.pakutaso.com (www.pakutaso.com)|133.242.30.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118905 (116K) [image/jpeg]
Saving to: ‘./test_img/src/test1.jpg’

./test_img/src/test 100%[===================>] 116.12K   237KB/s    in 0.5s    

2025-12-11 03:06:06 (237 KB/s) - ‘./test_img/src/test1.jpg’ saved [118905/118905]

/content/pulse
test1.jpg: Number of faces detected: 1


In [10]:
%cd /content/SemanticStyleGAN
!cp visualize/invert.py invert.py

# Fix for PyTorch 2.6+ UnpicklingError: Weights only load failed
!sed -i "s/ckpt = torch.load(args.ckpt)/ckpt = torch.load(args.ckpt, weights_only=False)/g" invert.py

!python invert.py \
  --ckpt pretrained/CelebAMask-HQ-512x512.pt \
  --imgdir test_img/align \
  --outdir results/inversion \
  --size 512

/content/SemanticStyleGAN
Namespace(ckpt='pretrained/CelebAMask-HQ-512x512.pt', imgdir='test_img/align', outdir='results/inversion', size=512, batch_size=1, no_noises=True, w_plus=True, save_steps=False, truncation=1, lr=0.1, lr_g=0.0001, step=400, finetune_step=0, noise_regularize=10, lambda_mse=0.1, lambda_lpips=1.0, lambda_mean=1.0)
Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 8,
 'channel_multiplier': 2,
 'checkpoint_dir': 'output/celeba-512',
 'ckpt': None,
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': 'dataset/lmdb_celebamaskhq_512',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'g_reg_every': 4,
 'inception': 'cache/cache_fid_celebahq_512_new.pkl',
 'iter': 800000,
 'lambda_mask': 100.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 4,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,


### ビット文字画像の取得

In [11]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/BitMoji-512x512.pt'
args.latent = './results/inversion/latent/test1_0.npy'
args.outdir = './results/style_BitMoji'
args.truncation = 0.7
args.truncation_mean = 10000
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt, weights_only=False)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])

/content/SemanticStyleGAN
Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 8,
 'channel_multiplier': 2,
 'checkpoint_dir': 'checkpoint/domain_adaptation/bitmoji_all/',
 'ckpt': 'checkpoint/cleaned/6827534-120000.pt',
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': 'dataset/lmdb_bitmoji_512',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'freeze_local': False,
 'g_reg_every': 4,
 'iter': 2000,
 'lambda_mask': 0.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 2,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1': 10,
 'residual_refine': True,
 'save_every': 100,
 'seg_dim': 13,
 'size': 512,
 'start_iter': 0,
 'transparent_dims': (10, 12),
 'viz_every': 100}
n_latent: 28, n_latent_expand: 130


  warnings.warn(



Generating original image ...


### MetFaces画像の取得

In [12]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/MetFaces-512x512.pt'
args.latent = './results/inversion/latent/test1_0.npy'
args.outdir = './results/style_MetFaces'
args.truncation = 0.7
args.truncation_mean = 10000
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt, weights_only=False)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])

/content/SemanticStyleGAN
Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 8,
 'channel_multiplier': 2,
 'checkpoint_dir': 'checkpoint/domain_adaptation/metfaces/',
 'ckpt': 'checkpoint/cleaned/6827534-120000.pt',
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': 'dataset/lmdb_metfaces_512',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'freeze_local': True,
 'g_reg_every': 4,
 'iter': 2001,
 'lambda_mask': 0.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 2,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1': 10,
 'residual_refine': True,
 'save_every': 200,
 'seg_dim': 13,
 'size': 512,
 'start_iter': 0,
 'transparent_dims': (10, 12),
 'viz_every': 100}
n_latent: 28, n_latent_expand: 130
Generating original image ...


In [13]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/Toonify-512x512.pt'
args.latent = './results/inversion/latent/test1_0.npy'
args.outdir = './results/style_Toonify'
args.truncation = 0.7
args.truncation_mean = 10000
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt, weights_only=False)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])

/content/SemanticStyleGAN
Loading model ...
Initializing model with arguments:
{'aug': False,
 'base_layers': 2,
 'batch': 8,
 'channel_multiplier': 2,
 'checkpoint_dir': 'checkpoint/domain_adaptation/toonify/',
 'ckpt': 'checkpoint/cleaned/6827534-120000.pt',
 'coarse_channel': 512,
 'coarse_size': 64,
 'd_reg_every': 16,
 'dataset': '/mnt/bd/yshi-dataset-generation-lq//face_dataset/MegaCartoon/lmdb_mega_512/',
 'depth_layers': 6,
 'detach_texture': False,
 'distributed': True,
 'freeze_local': True,
 'g_reg_every': 4,
 'iter': 1001,
 'lambda_mask': 0.0,
 'latent': 512,
 'local_channel': 64,
 'local_layers': 10,
 'local_rank': 0,
 'lr': 0.002,
 'min_feat_size': 16,
 'mixing': 0.3,
 'n_gpu': 2,
 'n_mlp': 8,
 'n_sample': 16,
 'num_workers': 8,
 'path_batch_shrink': 2,
 'path_regularize': 0.5,
 'r1': 10,
 'residual_refine': True,
 'save_every': 200,
 'seg_dim': 13,
 'size': 512,
 'start_iter': 0,
 'transparent_dims': (10, 12),
 'viz_every': 100}
n_latent: 28, n_latent_expand: 130
Generat

結果はresultsディレクトリの中に格納されているので確認するとよい

## 元の顔画像

In [ ]:
import os
from IPython.display import Image, display

# User specified path
target_path = '/content/SemanticStyleGAN/results/style_MetFaces/image.jpeg'

if os.path.exists(target_path):
    print(f"Displaying image from: {target_path}")
    display(Image(target_path))
else:
    print(f"Image not found at {target_path}\n")

## 絵文字風に変換

In [ ]:
import os
from IPython.display import Image, display

# User specified path
target_path = '/content/SemanticStyleGAN/results/style_BitMoji/image.jpeg'

if os.path.exists(target_path):
    print(f"Displaying image from: {target_path}")
    display(Image(target_path))
else:
    print(f"Image not found at {target_path}\n")

## マンガ調に変換

In [ ]:
import os
from IPython.display import Image, display

# User specified path
target_path = '/content/SemanticStyleGAN/results/style_Toonify/image.jpeg'

if os.path.exists(target_path):
    print(f"Displaying image from: {target_path}")
    display(Image(target_path))
else:
    print(f"Image not found at {target_path}\n")

# DragGAN(対話的Image Manipulation GAN)

DragGANは、画像の任意の点をドラッグし、ユーザーと対話的な方法でGANを制御することで画像を操作するImage Manipulation技術の一つ

プロンプトなどで画像を操作するのが一般的であるが、マウスなどを用いて与えた注目点と目標点(複数でもよい)について、注目点が目標点に向かって矛盾なくスムーズに移動するように画像を生成する

- 画像上のいくつかのハンドルポイント(赤)とターゲットポイント(青)をクリックすれば、対応するターゲットポイントに正確に到達するように操作点が移動する
- オプションで、画像の残りの部分を固定したまま、フレキシブルな領域(明るい領域)のマスクを指定できる

以下に例を示す

<img src="http://class.west.sd.keio.ac.jp/dataai/text/draggansamples.jpg" width=900>

実際の操作はつぎのようなイメージである

<img src="http://class.west.sd.keio.ac.jp/dataai/text/DragGAN.gif" width=600>



## 構造



特徴ベースの動き監視(feature-based motion supervision)と、ポイントをローカライズし続けるための点追跡(point tracking approach)の2つの主要コンポーネントにより柔軟な画像操作を実現している

処理の流れは次のとおりである

- GANで生成された画像が与えられ、ユーザーはいくつかの操作点(赤い点)、ターゲットポイント(青い点)、オプションとして編集中の可動領域を示すマスク(明るい領域を設定する

- 動き監視と点追跡を繰り返し実行する
  - 動き監視では操作点(赤い点)をターゲットポイント(青い点)に向かって移動させる
  - 点追跡では画像内のオブジェクトを追跡するために操作点を更新する
  
<img src="http://class.west.sd.keio.ac.jp/dataai/text/dragganflow.jpg" width=900>

GANの学習された生成画像多様体上(潜在空間内)で移動が実行されるため、オクルージョン(手前の物体で後ろの物体が隠れるような状況)されたコンテンツや、物体の剛性に一貫して従う形状の変形でも、現実的な出力を生成しようとする

<img src="http://class.west.sd.keio.ac.jp/dataai/text/dragganarch.jpg" width=900>

動作可能なデモも存在するが、現状のColab環境や、関連ライブラリの進捗スピードが速いため、すでにdepricateされている。

# チェックポイント

一度ここで中断し、次のセルから再度実行すること

# SadTalker(画像のリップシンク)

顔画像と音声の一部から自然に会話する動画を構築しようとすると、不自然な頭の動きや、歪んだ表情、アイデンティティの修正など、多くの課題がある

この問題の一因は、結合された2次元運動場から学習した場合、実際には3次元であることから生じているといえる

しかしながら、3次元情報を明示的に利用しても、表情が硬く、映像に連続性がなくなり不自然になるという問題が残る場合がある

そこで、SadTalkerは次の方針がとられている

- 音声から3DMM(3D Morphable Model)の3D動き係数(頭部ポーズ、表情)を生成し、3Dに基づく顔レンダリングによりトーキングヘッドを生成する

- 現実的な動き係数を学習するため、音声と様々なトーキングヘッドの動き係数の間の接続を個別かつ明示的にモデル化する

- この係数と3Dレンダリングされた顔の両方を蒸留することにより、音声から正確な表情を学習するExpNetを用いる
  - ここでモデルの蒸留とは、既存モデルの入力と出力のペアを元に新たなモデルの学習を行い、既存モデルとよく似たモデルを作成すること

- 頭部ポーズについて、条件付きVAEを介してPoseVAEを設計し、様々なスタイルの頭部モーションを合成する

- 最後に、生成された3D運動係数を、顔レンダリングの教師なし3Dキーポイント空間にマッピングして、映像を合成する

音声で顔画像を自然に動かすWebサービスD-IDと同様な機能を持つ

SadTalkerのパイプライン(Pipeline)を次の図に示す

<img src="http://class.west.sd.keio.ac.jp/dataai/text/sadtalker.png" width="600">

次の手順で処理を進める
- 顔画像(Single Input Image)から3Dの顔画像を再構築(Monocular 3D Face Recon)し、表情係数(Expression Coefficients)と頭のポーズ係数(Head Pose Coefficients)を抽出する
- 音声(Input Audio)から頭のポーズを学習するネットワーク(PoseVAE)と表情を学習するネットワーク(ExpNet)を用いて、これらの係数がどう変化するのかを求める
- 係数を元に3D対応の顔のレンダリング(3D-Aware Face Render)で合成されフレーム(Generated Frames)を生成する



## 環境構築


In [50]:
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
!python --version
!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

print('Git clone project and install requirements...')
!git clone https://github.com/cedro3/SadTalker.git &> /dev/null
%cd SadTalker
!export PYTHONPATH=/content/SadTalker:$PYTHONPATH
!python3.8 -m pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
!apt update
!apt install ffmpeg &> /dev/null
!python3.8 -m pip install -r requirements.txt


update-alternatives: error: alternative path /usr/bin/python3.8 doesn't exist
update-alternatives: error: alternative path /usr/bin/python3.9 doesn't exist
Python 3.12.12
Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/

In [51]:
!pip install safetensors
!pip install kornia
!pip install facexlib
!pip install yacs
!pip install pydub
!pip install gfpgan

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 89.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 3.50.2 requires numpy~=1.0, but you have numpy 2.0.2 which is incompatible.
yfinance 0.2.66 requires websockets>=13.0, but you have websockets 11.0.3 which is incompatible.


## モデルのダウンロード

In [52]:
print('Download pre-trained models...')
!rm -rf checkpoints
!bash scripts/download_models.sh

Download pre-trained models...
--2025-12-11 07:28:07--  https://github.com/OpenTalker/SadTalker/releases/download/v0.0.2-rc/mapping_00109-model.pth.tar
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/569518584/ccc415aa-c6f4-47ee-8250-b10bf440ba62?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-11T08%3A11%3A52Z&rscd=attachment%3B+filename%3Dmapping_00109-model.pth.tar&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-11T07%3A11%3A01Z&ske=2025-12-11T08%3A11%3A52Z&sks=b&skv=2018-11-09&sig=CWJIiRR26ENT52qe6kWcuJ0P1R2nGiS6gbIhwjnk88o%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTQ0MTY4NywibmJmIjo

## 推論

バージョン間の齟齬があり、そのままでは動作しないためパッチをあてる

In [53]:
# Fix basicsr import error (adjusting path for python 3.12)
!sed -i 's/from torchvision.transforms.functional_tensor import rgb_to_grayscale/from torchvision.transforms.functional import rgb_to_grayscale/' /usr/local/lib/python3.12/dist-packages/basicsr/data/degradations.py

# Fix facexlib numpy float error (adjusting path for python 3.12)
!sed -i 's/np.float/float/' /usr/local/lib/python3.12/dist-packages/facexlib/alignment/awing_arch.py

# Fix numpy array dtype object issue
!sed -iE '/trans_params = np.array/s/)$/, dtype=object)/' src/face3d/util/preprocess.py

# Fix numpy VisibleDeprecationWarning issue (removed in newer numpy)
!sed -i 's/warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)/# warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)/' src/face3d/util/preprocess.py

In [54]:
image ='full3.png'
audio ='eluosi.wav'
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --enhancer gfpgan

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00,  9.30it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 14.05it/s]
mel:: 100% 154/154 [00:00<00:00, 44111.37it/s]
audio2exp:: 100% 16/16 [00:00<00:00, 197.57it/s]
Face Renderer:: 100% 77/77 [00:46<00:00,  1.67it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 259) to (256, 272) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
The generated video is named ./results/2025_12_11_07.28.59/full3##eluosi.mp4
face enhancer....
Face Enhancer:: 100% 154/154 [00:50<00:00,  3.03it/s]
The generated video is named ./results/2025_12_11_07.28.59/full3##eluosi_enhanced.mp4
The generated video is named: ./results/2025_12_11_07.28.59.mp4


## 動画の再生

In [55]:
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# Check if any mp4 files exist
mp4_files = glob.glob('./results/*.mp4')
if mp4_files:
    # get the last from results
    mp4_name = sorted(mp4_files)[-1]

    mp4 = open('{}'.format(mp4_name),'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    print('Display animation: {}'.format(mp4_name), file=sys.stderr)
    display(HTML("""
      <video width=256 controls>
            <source src="%s" type="video/mp4">
      </video>
      """ % data_url))
else:
    print("No mp4 files found in './results'. Check if 'inference.py' ran successfully and generated output.")
    # List the contents of the results directory for debugging
    print("Contents of './results':", os.listdir('./results'))

Display animation: ./results/2025_12_11_07.28.59.mp4


## 推論(日本語)

imageに画像ファイル名を、audioに音声ファイル名を指定する

In [56]:
image ='full3.png'
audio ='japanese.wav'
#audio ='itosinger1.wav'
#audio ='eluosi.wav'
#audio ='RD_Radio31_000.wav'
#audio ='RD_Radio34_002.wav'
#audio ='RD_Radio36_000.wav'
#audio ='RD_Radio40_000.wav'
source_image = 'examples/source_image/' + image
driven_audio = 'examples/driven_audio/' + audio

!python inference.py --driven_audio $driven_audio \
           --source_image $source_image \
           --result_dir ./results --still --preprocess full --enhancer gfpgan

using safetensor as default
3DMM Extraction for source image
landmark Det:: 100% 1/1 [00:00<00:00,  9.61it/s]
3DMM Extraction In Video:: 100% 1/1 [00:00<00:00, 20.09it/s]
mel:: 100% 227/227 [00:00<00:00, 44441.14it/s]
audio2exp:: 100% 23/23 [00:00<00:00, 254.28it/s]
Face Renderer:: 100% 114/114 [01:10<00:00,  1.61it/s]
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (256, 259) to (256, 272) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
The generated video is named ./results/2025_12_11_07.31.09/full3##japanese.mp4
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
seamlessClone:: 100% 227/227 [00:37<00:00,  6.01it/s]
The generated video is named ./results/2025_12_11_07.31.09/f

## 動画の再生(日本語)

In [57]:
import glob
from IPython.display import HTML
from base64 import b64encode
import os, sys

# get the last from results
mp4_name = sorted(glob.glob('./results/*.mp4'))[-1]

mp4 = open('{}'.format(mp4_name),'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

print('Display animation: {}'.format(mp4_name), file=sys.stderr)
display(HTML("""
  <video width=256 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

Display animation: ./results/2025_12_11_07.31.09.mp4


## 課題

各自、画像ファイルと、音声ファイルを準備し、リップシンクさせなさい

# チェックポイント

一度ここで中断し、次のセルから再度実行すること

In [58]:
from google.colab import runtime
runtime.unassign()

# Wav2Lip(動画のリップシンク)

Wav2Lipはlip-syncingと呼ばれる、任意のビデオに映る人物の唇の動きを、任意の音声に一致させる技術の一実装である

静止画であれば、座標軸を動かしてモーフィングさせることは容易であるが、動的で制約のない会話中の顔のビデオで同様のことを行うためには、唇の動きそのものを正確に捉えて、その都度正しくモーフィングさせることが必要となるため、違和感なく音声と同期させることが困難となる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/wav2lip1.jpg" width=900>

Wav2Lipでは、事前に訓練されたPre-trained Lip-Sync Expertを基底として学習させることで正確なリップシンクを生成するとともに、やはり事前訓練されたDiscriminatorを用いてリップシンクエラーを検出し修正することで視覚品質を向上させる

リップシンクエラーを検出する際には、事前に訓練された識別器を使用するが、ノイズを含めて生成した顔でさらに微調整を行うと、識別器のリップシンク測定能力が低下し、生成される唇の形状にも影響を与えることができる

さらに、視覚的品質識別器により同期精度とともに視覚的品質を向上させる

<img src="http://class.west.sd.keio.ac.jp/dataai/text/wav2lip2.jpg" width=900>


## 環境構築

コードツリーを取得する

In [76]:
version = 'v8.3'

#check if already installed
import os
import sys
if os.path.exists('installed.txt'):
  with open('last_file.txt', 'r') as file:
    last_file = file.readline()
  if last_file == version:
    sys.exit('Easy-Wav2Lip '+version+' has already been run on this instance!')

#check GPU is enabled
print('checking for GPU')
import torch
if not torch.cuda.is_available():
  sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')

#start timer
import time
start_time = time.time()

#clone git
giturl = 'https://github.com/anothermartz/Easy-Wav2Lip.git'

!git clone -b {version} {giturl}
%cd 'Easy-Wav2Lip'
working_directory = os.getcwd()
!mkdir 'face_alignment' 'temp'

#install prerequisites
print('installing batch_face')
import warnings
warnings.filterwarnings("ignore", category=UserWarning,
                        module='torchvision.transforms.functional_tensor')
!pip install batch_face --quiet
!pip install basicsr==1.4.2 --quiet
print('fixing basicsr degradations.py')

# Dynamically get python version to fix path issue
python_version = f"{sys.version_info.major}.{sys.version_info.minor}"
!cp /content/Easy-Wav2Lip/degradations.py /usr/local/lib/python{python_version}/dist-packages/basicsr/data/degradations.py

print('installing gfpgan')
!pip install gfpgan --quiet

!python install.py

from IPython.display import clear_output
clear_output()
print("Installation complete, move to Step 2!")

#end timer
elapsed_time = time.time() - start_time
from easy_functions import format_time
print(f"Execution time: {format_time(elapsed_time)}")

FileNotFoundError: [Errno 2] No such file or directory: 'last_file.txt'

In [ ]:
%cd /content/DragGAN

print("Applying definitive patches...")

# 1. Force-patch gradio_utils/utils.py using sed
# Replaces the __init__ line with one that pops conflicting kwargs immediately.
!sed -i 's/def __init__(self, \*\*kwargs):/def __init__(self, **kwargs): kwargs.pop("interactive", None); kwargs.pop("source", None); kwargs.pop("tool", None);/g' gradio_utils/utils.py

# 2. Ensure dependencies are correct
!pip install gradio==3.50.2 "numpy<2" --quiet

# 3. Ensure other patches are applied (just in case)
!sed -i 's/torch\.load(f, map_location=map_location)/torch.load(f, map_location=map_location, weights_only=False)/' legacy.py
!sed -i 's/return torch\.load(f)/return torch.load(f, weights_only=False)/' legacy.py
!sed -i "s/impl='cuda'/impl='ref'/g" torch_utils/ops/bias_act.py
!sed -i "s/impl='cuda'/impl='ref'/g" torch_utils/ops/upfirdn2d.py
!sed -i 's/text_width, text_height = font.getsize(watermark_text)/left, top, right, bottom = font.getbbox(watermark_text); text_width = right - left; text_height = bottom - top/' viz/renderer.py
!sed -i 's/Image\.ANTIALIAS/Image.LANCZOS/g' gradio_utils/utils.py

print("Starting DragGAN GUI...")
!python visualizer_drag_gradio.py

In [ ]:
!pip install moviepy imageio
!pip install librosa

In [ ]:
from moviepy.video.fx.resize import resize
from moviepy.editor import VideoFileClip
from IPython.display import Audio, display

## テスト動画&音声の準備

各自で準備するとよい

## 入力動画の取得

In [ ]:
video_url = 'https://www.youtube.com/watch?v=-Xvpo0A5X6g' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  30#@param {type:"integer"}
end_sec =  35#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

## 動画のクリッピング

In [ ]:
!mkdir input_video

In [ ]:
import os
if not os.path.exists('input_video/full_video.mp4'):
  !gdown "10om2n4HgyLTk-hA9oDmeEK0BZ4EA2KE9" -O input_video/full_video.mp4


In [ ]:
download_resolution = 720
full_video_path = './input_video/full_video.mp4'
input_clip_path = './input_video/clip_video.mp4'

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

In [ ]:
# 動画の確認
clip = VideoFileClip(input_clip_path)
clip = resize(clip, height=420)
clip.ipython_display()

## 入力音声の取得

In [ ]:
video_url = 'https://www.youtube.com/watch?v=fD-yyMzF8lI' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  63#@param {type:"integer"}
end_sec =  68#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

In [ ]:
!mkdir input_audio

In [ ]:
if not os.path.exists('input_audio/full_audio.mp4'):
  !gdown "10ys5h4sCuqg9ssNOLJZ4jy8F4ptMJn9V" -O input_audio/full_audio.mp4

In [ ]:
download_resolution = 720
full_video_path = './input_audio/full_audio.mp4'
input_clip_path = './input_audio/clip_audio.mp4'
input_audio_path = './input_audio/clip_audio.mp3'

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(input_clip_path)

# 音声抽出
videoclip = VideoFileClip(input_clip_path)
audioclip = videoclip.audio
audioclip.write_audiofile(input_audio_path)

In [ ]:
display(Audio(input_audio_path, autoplay=True))

In [ ]:
if not os.path.exists('installed.txt'):
  sys.exit('Step 1 has not been run in this instance! Please run step 1 each time you disconnect from a runtime.')
time
############################## user inputs #####################################
#@markdown <h1>Step 2: Select inputs:</h1>

# @markdown On destktop: <h1></h1>Click the folder icon ( 📁 ) at the left edge of colab, find your file, right click, copy path, paste it below:
#@markdown<br></br>
# @markdown On mobile: <h1></h1>Tap the hamburger button ( ☰ ) at the top left, click show file browser, find your file, long press on it, copy path, paste below:
video_file = "/content/Easy-Wav2Lip/input_video/clip_video.mp4" #@param {type:"string"}
vocal_file = "/content/Easy-Wav2Lip/input_audio/clip_audio.mp3" #@param {type:"string"}

#@markdown > Keep vocal_file blank if your video already has the desired speech audio encoded into it.
#@markdown # Quality
quality = "Enhanced" # @param ["Fast", "Improved", "Enhanced"]
#@markdown * <b><u>Fast</u></b>: Wav2Lip <br>
#@markdown * <b><u>Improved</u></b>: Wav2Lip with a feathered mask around the mouth to remove the square around the face <br>
#@markdown * <b><u>Enhanced</u></b>: Wav2Lip + mask + GFPGAN upscaling done on the face
#preview_quality = False #@param {type:"boolean"} - coming soon!
output_height = "full resolution" #@param ["half resolution", "full resolution", "480"] {allow-input: true}
use_previous_tracking_data = True #@param {type:"boolean"}
#@markdown Speeds up processing of the same video used multiple times - it should delete the last tracking file automatically when the video is changed but if it's failing after the first video, untick this box.

#@markdown
#------------------------------*Step 3*----------------------------------------!
#@markdown <h1>👈 Step 3:  Click the little circle play button on this cell! </h1> (Or press ctrl + F10) - Then wait for processing to complete.
# scale padding with resolution
#@markdown <br>

#@markdown ---
#@markdown <br>

#@markdown # [Advanced tweaking](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#advanced-tweaking) (optional) </h1>Just ignore all of this if you are new, or click the blue titles for instructions.
wav2lip_version = "Wav2Lip" # @param ["Wav2Lip", "Wav2Lip_GAN"]
nosmooth = True #@param {type:"boolean"}
#@markdown ### [Padding:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#padding)</h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-100, max:100, step:1}
D = 10 #@param {type:"slider", min:-100, max:100, step:1}
L = 0 #@param {type:"slider", min:-100, max:100, step:1}
R = 0 #@param {type:"slider", min:-100, max:100, step:1}
#@markdown <br>

#@markdown ### [Mask:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)
size = 1.5 #@param {type:"slider", min:1, max:6, step:0.1}
feathering = 1 #@param {type:"slider", min:0, max:3, step:1}
mouth_tracking = False #@param {type:"boolean"}
debug_mask = False #@param {type:"boolean"}


#@markdown # [Other options:](https://github.com/anothermartz/Easy-Wav2Lip/tree/v7#other-options)

batch_process = False #@param {type:"boolean"}
output_suffix = "_Easy-Wav2Lip" #@param {type:"string"}
include_settings_in_suffix = False #@param {type:"boolean"}
preview_input = False #@param {type:"boolean"}
preview_settings = False #@param {type:"boolean"}
#@markdown preview_settings processes only one frame so you can see how it looks without doing the whole video
frame_to_preview = 100 # @param {type:"integer"}


import configparser

# Create a ConfigParser object
config = configparser.ConfigParser()

# Put all your variables in a dictionary
options = {
    'video_file': video_file,
    'vocal_file': vocal_file,
    'quality': quality,
    'output_height': output_height,
    'wav2lip_version': wav2lip_version,
    'use_previous_tracking_data': use_previous_tracking_data,
    'nosmooth': nosmooth
}
padding = {
    'U': U,
    'D': D,
    'L': L,
    'R': R
}
mask = {
    'size': size,
    'feathering': feathering,
    'mouth_tracking': mouth_tracking,
    'debug_mask': debug_mask
}
other = {
    'batch_process': batch_process,
    'output_suffix': output_suffix,
    'include_settings_in_suffix': include_settings_in_suffix,
    'preview_input': preview_input,
    'preview_settings': preview_settings,
    'frame_to_preview': frame_to_preview
}


# Add the dictionary to the ConfigParser object
config['OPTIONS'] = options
config['PADDING'] = padding
config['MASK'] = mask
config['OTHER'] = other

# Write the data to an INI file
with open('config.ini', 'w') as f:
    config.write(f)

In [ ]:
!python run.py

from easy_functions import show_video
from IPython.display import Image
if preview_settings:
  if os.path.isfile(os.path.join('temp','preview.jpg')):
    display(Image(os.path.join('temp','preview.jpg')))
else:
  if os.path.isfile(os.path.join('temp','output.mp4')):
    print(f"Loading video preview...")
    show_video(os.path.join('temp','output.mp4'))